In [ ]:
!pip install streamlit
!pip install networkx
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
import torch

from transformers import AutoTokenizer
model_name = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

path = f"/content/drive/MyDrive/QA_System/Models/ROBERTA/xlm_roberta_large5.pt"

model=torch.load(path,map_location=torch.device('cpu'))
model.eval()

# Function to predict answer based on provided context and query
def predict(context,query):

  # Encode query and context 
  inputs = tokenizer.encode_plus(query, context, padding=True, max_length= 512, truncation=True, add_special_tokens = True, return_tensors='pt')

  # Generate output
  outputs = model(**inputs)

  # Extract start and end index 
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1 

  # Decode answer from tokenized form
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

# Normalize text -> Remove articles, spaces, punctuation marks and conversion to all upper cases to lower cases
def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

# Compute Exact Match Score (em score)
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

#Compute F1 score
def compute_f1(prediction, truth):
  # Normalizing predicted and true answer
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  # Intersection of Predicted and true answer
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  # Ratio of length of common tokens to complete tokens length
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  # Return Harmonic Mean
  return 2 * (prec * rec) / (prec + rec)

# Predicting answer and returning metrics of evaluation 
def give_an_answer(context,query,answer):

  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  
  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print("\n")

  return prediction

st.title("Multilingual Question Answering System")
para = st.text_input("Enter Paragraph")
ques = st.text_input("Enter your Question")


context = str(para)
queries = str(ques)

answer = "cricket"


if(st.button("Predict")):
    if(queries=="" or context==""):
      st.error("Fill the details Correctly")
    
    else :
      answer = give_an_answer(context, queries, answer)
      st.success('Predicted answer is {}'.format(answer))

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501